In [41]:
import numpy as np
#from utils2 import *
from tensorflow.keras.layers import *
from tensorflow.keras import *
import tensorflow as tf
import scipy.io as sio

In [42]:
# transfer the phase to complex-valued analog beamformer
def trans_Vrf(temp):
    v_real = tf.cos(temp)
    v_imag = tf.sin(temp)
    vrf = tf.cast(tf.complex(v_real, v_imag), tf.complex64)
    return vrf

def Rate_func(temp):
    h, v, SNR_input = temp
    hv = tf.keras.backend.batch_dot(
        tf.cast(h, tf.complex64), v)
    rate = tf.math.log(tf.cast(1 + SNR_input / Nt * tf.pow(tf.abs(hv), 2), tf.float32)) / tf.math.log(2.0)
    return -rate

def mat_load(path):
    print('loading data...')
    # load the perfect csi
    h = sio.loadmat(path + '\pcsi.mat')['pcsi']
    # load the estimated csi
    h_est = sio.loadmat(path + '\ecsi.mat')['ecsi']
    print('loading complete')
    print('The shape of CSI is: ', h_est.shape)
    return h, h_est


In [43]:
H_est=sio.loadmat(r"C:\Users\Jamal\OneDrive\Jamal\MATLAB Code\Simulation Codes of Papers\Beamforming design for large-scale antenna arrays using deep learning\BF-design-with-DL-master\ecsi.mat")
H_est=H_est['ecsi']
H=sio.loadmat(r"C:\Users\Jamal\OneDrive\Jamal\MATLAB Code\Simulation Codes of Papers\Beamforming design for large-scale antenna arrays using deep learning\BF-design-with-DL-master\pcsi.mat")
H=H['pcsi']

# use the estimated csi as the input of the BFNN
H_input = np.expand_dims(np.concatenate([np.real(H_est), np.imag(H_est)], 1), 1)
print(H_input.shape)

# H denotes the perfect csi
H = np.squeeze(H)
print(H.shape)

# generate  SNRs associated with different samples
SNR = np.power(10, np.random.randint(-20, 20, [H.shape[0], 1]) / 10)

print(SNR.shape)

Nt=64


(10000, 1, 2, 64)
(10000, 64)
(10000, 1)


In [44]:
# -----------------------
#  Construct the BFNN Model
# -----------------------
# imperfect CSI is used to output the vrf
imperfect_CSI = Input(name='imperfect_CSI', shape=(H_input.shape[1:4]), dtype=tf.float32)
# perfect_CSI is only used to compute the loss, and not required in prediction
perfect_CSI = Input(name='perfect_CSI', shape=(H.shape[1],), dtype=tf.complex64)
# the SNR is also fed into the BFNN
SNR_input = Input(name='SNR_input', shape=(1,), dtype=tf.float32)
temp = BatchNormalization()(imperfect_CSI)
temp = Flatten()(temp)
temp = BatchNormalization()(temp)
temp = Dense(256, activation='relu')(temp)
temp = BatchNormalization()(temp)
temp = Dense(128, activation='relu')(temp)
phase = Dense(Nt)(temp)
V_RF = Lambda(trans_Vrf, dtype=tf.complex64, output_shape=(Nt,))(phase)
rate = Lambda(Rate_func, dtype=tf.float32, output_shape=(1,))([perfect_CSI, V_RF, SNR_input])
model = Model(inputs=[imperfect_CSI, perfect_CSI, SNR_input], outputs=rate)
# the y_pred is the actual rate, thus the loss is y_pred, without labels
model.compile(optimizer='adam', loss=lambda y_true, y_pred: y_pred)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 imperfect_CSI (InputLayer)     [(None, 1, 2, 64)]   0           []                               
                                                                                                  
 batch_normalization_12 (BatchN  (None, 1, 2, 64)    256         ['imperfect_CSI[0][0]']          
 ormalization)                                                                                    
                                                                                                  
 flatten_4 (Flatten)            (None, 128)          0           ['batch_normalization_12[0][0]'] 
                                                                                                  
 batch_normalization_13 (BatchN  (None, 128)         512         ['flatten_4[0][0]']        

In [45]:
# -----------------------
#  Train Your Model
# -----------------------
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=20, min_lr=0.00005)
checkpoint = callbacks.ModelCheckpoint('./temp_trained.h5', monitor='val_loss',
                                       verbose=0, save_best_only=True, mode='min', save_weights_only=True)
model.fit(x=[H_input, H, SNR], y=H, batch_size=256,
          epochs=50, verbose=2, validation_split=0.1, callbacks=[reduce_lr, checkpoint])


Epoch 1/50
36/36 - 1s - loss: -2.8975e+00 - val_loss: -2.3839e+00 - lr: 0.0010 - 903ms/epoch - 25ms/step
Epoch 2/50
36/36 - 0s - loss: -3.8405e+00 - val_loss: -2.9612e+00 - lr: 0.0010 - 158ms/epoch - 4ms/step
Epoch 3/50
36/36 - 0s - loss: -4.1251e+00 - val_loss: -3.4653e+00 - lr: 0.0010 - 161ms/epoch - 4ms/step
Epoch 4/50
36/36 - 0s - loss: -4.2800e+00 - val_loss: -3.8298e+00 - lr: 0.0010 - 176ms/epoch - 5ms/step
Epoch 5/50
36/36 - 0s - loss: -4.3767e+00 - val_loss: -4.0835e+00 - lr: 0.0010 - 188ms/epoch - 5ms/step
Epoch 6/50
36/36 - 0s - loss: -4.4430e+00 - val_loss: -4.2352e+00 - lr: 0.0010 - 177ms/epoch - 5ms/step
Epoch 7/50
36/36 - 0s - loss: -4.4980e+00 - val_loss: -4.3225e+00 - lr: 0.0010 - 173ms/epoch - 5ms/step
Epoch 8/50
36/36 - 0s - loss: -4.5409e+00 - val_loss: -4.3787e+00 - lr: 0.0010 - 168ms/epoch - 5ms/step
Epoch 9/50
36/36 - 0s - loss: -4.5763e+00 - val_loss: -4.4205e+00 - lr: 0.0010 - 167ms/epoch - 5ms/step
Epoch 10/50
36/36 - 0s - loss: -4.5995e+00 - val_loss: -4.4479e

In [50]:
# -----------------------
#  Test Your Model
# -----------------------

#model.load_weights(r'C:\Users\Jamal\OneDrive\Jamal\MATLAB Code\Simulation Codes of Papers\Beamforming design for large-scale antenna arrays using deep learning\BF-design-with-DL-master')

H_etest=sio.loadmat(r"C:\Users\Jamal\OneDrive\Jamal\MATLAB Code\Simulation Codes of Papers\Beamforming design for large-scale antenna arrays using deep learning\BF-design-with-DL-master\ecsit.mat")
H_etest=H_etest['ecsi']
#print(H_etest.shape)

H_t=sio.loadmat(r"C:\Users\Jamal\OneDrive\Jamal\MATLAB Code\Simulation Codes of Papers\Beamforming design for large-scale antenna arrays using deep learning\BF-design-with-DL-master\pcsit.mat")
H_t=H_t['pcsi']
H_t = np.squeeze(H_t)
#print(H_t.shape)

# use the estimated csi as the input of the BFNN
H_etest = np.expand_dims(np.concatenate([np.real(H_etest), np.imag(H_etest)], 1), 1)
#print(H_input.shape)
SNR = np.power(10, np.ones([H_t.shape[0], 1]) * 10 / 10)
Rates=model([H_etest, H_t, SNR])
print(np.mean(Rates.numpy()))


SNR = np.power(10, np.ones([H.shape[0], 1]) * 10 / 10)
Rates=model([H_input, H, SNR])

print(np.mean(Rates.numpy()))
# use the estimated csi as the input of the BFNN
#H_input = np.expand_dims(np.concatenate([np.real(H_est), np.imag(H_est)], 1), 1)

-7.496137
-7.8621893


In [ ]:
rate = []
# model.load_weights('./0db.h5')
for snr in range(-20, 25, 5):
    SNR = np.power(10, np.ones([H.shape[0], 1]) * snr / 10)
    y = model.evaluate(x=[H_input, H, SNR], y=H, batch_size=10000)
    print(snr, y)
    rate.append(-y)
print(rate)